# Dataset Source

https://www.kaggle.com/datasets/artyomkruglov/gaming-profiles-2025-steam-playstation-xbox

Note: Before continue, please download the dataset and extract all 3 dataset folders inside the archive to the same folder of the notebook file.

# Import Libraries

In [1]:
# For data manipulation and analysis
import pandas as pd
# For working with arrays
import numpy as np
# For working with database
import mysql.connector
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column, \
relationship, Session
from sqlalchemy import create_engine, ForeignKey
# Ref: https://docs.sqlalchemy.org/en/20/dialects/mysql.html#mysql-data-types
from sqlalchemy.dialects.mysql import (
    BIGINT,
    MEDIUMINT,
    SMALLINT,
    INTEGER,
    FLOAT,
    DATETIME,
    TEXT,
    MEDIUMTEXT,
    NVARCHAR,
    VARCHAR,
    DATE
)
from sqlalchemy.dialects.mysql import insert
# For working with types
from typing import List
from datetime import datetime

# Config settings

In [2]:
# Enable copy on write behavior in pandas
# Ref: https://pandas.pydata.org/pandas-docs/stable/user_guide/copy_on_write.html#copy-on-write
pd.options.mode.copy_on_write = True

# Reads Data

## Reads PlayStation Data

In [196]:
# Read game data
playstation_games_df = pd.read_csv('playstation/games.csv')

In [197]:
# Read achievement data
playstation_achievements_df = pd.read_csv('playstation/achievements.csv')

In [198]:
# Read player data
playstation_players_df = pd.read_csv('playstation/players.csv')

In [199]:
# Read history data
playstation_history_df = pd.read_csv('playstation/history.csv')

In [200]:
# Read price data
playstation_prices_df = pd.read_csv('playstation/prices.csv')

In [201]:
# Read purchased game data
playstation_purchased_games_df = pd.read_csv('playstation/purchased_games.csv')

## Reads Steam Data

In [3]:
# Read game data
steam_games_df = pd.read_csv('steam/games.csv')

In [4]:
# Read achievement data
steam_achievements_df = pd.read_csv('steam/achievements.csv')

In [5]:
# Read player data
steam_players_df = pd.read_csv('steam/players.csv')

In [6]:
# Read friend data
steam_friends_df = pd.read_csv('steam/friends.csv')

In [7]:
# Read history data
steam_history_df = pd.read_csv('steam/history.csv')

In [8]:
# Read price data
steam_prices_df = pd.read_csv('steam/prices.csv')

In [9]:
# Read purchased game data
steam_purchased_games_df = pd.read_csv('steam/purchased_games.csv')

In [10]:
# Read review data
steam_reviews_df = pd.read_csv('steam/reviews.csv')

Note: exlcuded `private_steamids` data.

## Reads Xbox Data

In [3]:
# Read achievement data
xbox_achievements_df = pd.read_csv('xbox/achievements.csv')

In [4]:
# Read game data
xbox_games_df = pd.read_csv('xbox/games.csv')

In [5]:
# Read history data
xbox_history_df = pd.read_csv('xbox/history.csv')

In [6]:
# Read player data
xbox_players_df = pd.read_csv('xbox/players.csv')

In [7]:
# Read price data
xbox_prices_df = pd.read_csv('xbox/prices.csv')

In [8]:
# Read purchased game data
xbox_purchased_games_df = pd.read_csv('xbox/purchased_games.csv')

# Processes And Cleans Data

## PlayStation Data

### Game Data

#### Info

In [216]:
playstation_games_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23151 entries, 0 to 23150
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   gameid               23151 non-null  int64 
 1   title                23151 non-null  object
 2   platform             23151 non-null  object
 3   developers           23134 non-null  object
 4   publishers           23140 non-null  object
 5   genres               23009 non-null  object
 6   supported_languages  10614 non-null  object
 7   release_date         23151 non-null  object
dtypes: int64(1), object(7)
memory usage: 1.4+ MB


#### Trim Data

In [217]:
playstation_games_df['title'] = \
  playstation_games_df['title'].str.strip()

playstation_games_df['platform'] = \
  playstation_games_df['platform'].str.strip()

playstation_games_df['developers'] = \
  playstation_games_df['developers'].str.strip()

playstation_games_df['publishers'] = \
  playstation_games_df['publishers'].str.strip()

playstation_games_df['genres'] = \
  playstation_games_df['genres'].str.strip()

playstation_games_df['supported_languages'] = \
  playstation_games_df['supported_languages'].str.strip()

playstation_games_df['release_date'] = \
  playstation_games_df['release_date'].str.strip()

#### Format Data

Format release_date to `'%Y-%m-%d'`:

In [218]:
# Convert release_date from string to datetime
playstation_games_df['release_date'] = pd.to_datetime(
    playstation_games_df['release_date'],
    errors='coerce').dt.strftime('%Y-%m-%d')

#### Remove Duplicates

Check for games with duplicated title, platform, developers and publishers:

In [219]:
playstation_games_df[
    playstation_games_df.duplicated(
        subset=['title', 'platform', 'developers', 'publishers'],
        keep=False
    )
].sort_values(by=['title', 'platform', 'developers', 'publishers'])

,gameid,title,platform,developers,publishers,genres,supported_languages,release_date
3840,572291,"""Buy The Game, I Have a Gun"" -Sheesh-Man",PS4,"[""Kimulator's Films""]","[""Kimulator's Films""]",['Visual Novel'],NaN,2023-01-29
3842,572182,"""Buy The Game, I Have a Gun"" -Sheesh-Man",PS4,"[""Kimulator's Films""]","[""Kimulator's Films""]",['Visual Novel'],NaN,2023-01-29
11886,7728,#KILLALLZOMBIES,PS4,['Beatshapers'],['Beatshapers'],['shooter'],NaN,2014-10-28
11889,7724,#KILLALLZOMBIES,PS4,['Beatshapers'],['Beatshapers'],"['Action', 'shooter']",NaN,2014-10-28
4916,519164,'n Verlore Verstand,PS4,['Skobbejak Games'],['Skobbejak Games'],"['Puzzle', 'Adventure', 'Platformer']",NaN,2018-06-12
...,...,...,...,...,...,...,...,...
15582,652889,qomp 2,PS4,['Graphite Lab'],['Atari'],['Action'],NaN,2024-02-20
1828,653611,qomp 2,PS5,['Graphite Lab'],['Atari'],['Action'],NaN,2024-02-20
15556,653610,qomp 2,PS5,['Graphite Lab'],['Atari'],['Action'],NaN,2024-02-20
7591,401581,void tRrLM(); //Void Terrarium,PS4,['Nippon Ichi Software'],['nis america'],"['Roguelite', 'Role Playing', 'Turn Based', 'D...",NaN,2021-05-18


Remove duplicates:

In [220]:
playstation_games_df.drop_duplicates(
    subset=['title', 'platform', 'developers', 'publishers'],
    inplace=True
)

#### Remove Missing Values

Remove games with missing title:

In [221]:
playstation_games_df.dropna(subset=['title'], inplace=True)

View info again:

In [222]:
playstation_games_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14741 entries, 0 to 23149
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   gameid               14741 non-null  int64 
 1   title                14741 non-null  object
 2   platform             14741 non-null  object
 3   developers           14726 non-null  object
 4   publishers           14731 non-null  object
 5   genres               14629 non-null  object
 6   supported_languages  8219 non-null   object
 7   release_date         14741 non-null  object
dtypes: int64(1), object(7)
memory usage: 1.0+ MB


### Achievement Data

#### Info

In [223]:
playstation_achievements_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 846563 entries, 0 to 846562
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   achievementid  846563 non-null  object
 1   gameid         846563 non-null  int64 
 2   title          846559 non-null  object
 3   description    846547 non-null  object
 4   rarity         846563 non-null  object
dtypes: int64(1), object(4)
memory usage: 32.3+ MB


#### Trim Data

In [224]:
playstation_achievements_df['achievementid'] = \
  playstation_achievements_df['achievementid'].str.strip()

playstation_achievements_df['title'] = \
  playstation_achievements_df['title'].str.strip()

playstation_achievements_df['description'] = \
  playstation_achievements_df['description'].str.strip()

playstation_achievements_df['rarity'] = \
  playstation_achievements_df['rarity'].str.strip()

#### Filter Data

Filter achievements having games that only exist in `playstation_games_df`

In [225]:
playstation_achievements_df = playstation_achievements_df[
    playstation_achievements_df['gameid'].isin(playstation_games_df['gameid'])
]

#### About Duplicated Achievements

Players can **earn duplicate achievements** for the same game on PlayStation under certain conditions:

* Different Regions – Some games have separate trophy lists for different region (e.g., NA, EU, JP). If you play the same game but from different regions, you can earn trophies again.

* Different Platforms – If a game has separate trophy lists for PS4 and PS5 versions, you can earn the trophies twice (or more, depending on the platforms available).

* Stackable Trophy Lists – Some developers release multiple versions of the same game with separate trophy lists (e.g., standard edition vs. deluxe edition).

* Remastered or Re-released Versions – A remastered or re-released version of a game often has a new trophy list, allowing you to earn the same trophies again.

#### Remove Missing Values

Remove achievements with missing title:

In [226]:
playstation_achievements_df.dropna(subset=['title'], inplace=True)

View info again:

In [227]:
playstation_achievements_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 410720 entries, 0 to 846562
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   achievementid  410720 non-null  object
 1   gameid         410720 non-null  int64 
 2   title          410720 non-null  object
 3   description    410704 non-null  object
 4   rarity         410720 non-null  object
dtypes: int64(1), object(4)
memory usage: 18.8+ MB


### Player Data

#### Info

In [228]:
playstation_players_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 356600 entries, 0 to 356599
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   playerid  356600 non-null  int64 
 1   nickname  356600 non-null  object
 2   country   356600 non-null  object
dtypes: int64(1), object(2)
memory usage: 8.2+ MB


#### Trim Data

In [229]:
playstation_players_df['nickname'] = \
  playstation_players_df['nickname'].str.strip()

playstation_players_df['country'] = \
  playstation_players_df['country'].str.strip()

#### About Duplicated Player Nicknames

All nicknames must be unique.

Check for duplicated nicknames.

In [230]:
playstation_players_df[
    playstation_players_df.duplicated(
        subset=['nickname'],
        keep=False
    )
].sort_values(by=['nickname'])

,playerid,nickname,country


Looks good.

### History Data

#### Info

In [231]:
playstation_history_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19510083 entries, 0 to 19510082
Data columns (total 3 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   playerid       int64 
 1   achievementid  object
 2   date_acquired  object
dtypes: int64(1), object(2)
memory usage: 446.6+ MB


#### Trim Data

In [232]:
playstation_history_df['achievementid'] = \
  playstation_history_df['achievementid'].str.strip()

playstation_history_df['date_acquired'] = \
  playstation_history_df['date_acquired'].str.strip()

#### Convert Data

`date_acquired` should be converted to date time type:

In [233]:
playstation_history_df['date_acquired'] = pd.to_datetime(
    playstation_history_df['date_acquired'],
    errors='coerce').dt.strftime('%Y-%m-%d %H:%M:%S')

#### Filter Data

Filter history records that only exist in `playstation_achievements_df` and `playstation_players_df`

In [234]:
playstation_history_df = playstation_history_df[
    playstation_history_df['achievementid'].isin(
        playstation_achievements_df['achievementid']) &
    playstation_history_df['playerid'].isin(
        playstation_players_df['playerid'])
]

View info again:

In [235]:
playstation_history_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13454632 entries, 0 to 19510082
Data columns (total 3 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   playerid       int64 
 1   achievementid  object
 2   date_acquired  object
dtypes: int64(1), object(2)
memory usage: 410.6+ MB


### Price Data

#### Info

In [236]:
playstation_prices_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62816 entries, 0 to 62815
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   gameid         62816 non-null  int64  
 1   usd            49251 non-null  float64
 2   eur            47264 non-null  float64
 3   gbp            48637 non-null  float64
 4   jpy            27038 non-null  float64
 5   rub            10911 non-null  float64
 6   date_acquired  62816 non-null  object 
dtypes: float64(5), int64(1), object(1)
memory usage: 3.4+ MB


#### Trim Data

In [237]:
playstation_prices_df['date_acquired'] = \
  playstation_prices_df['date_acquired'].str.strip()

#### Convert Data

`date_acquired` should be converted to date time type:

In [238]:
playstation_prices_df['date_acquired'] = pd.to_datetime(
    playstation_prices_df['date_acquired'],
    errors='coerce').dt.strftime('%Y-%m-%d')

#### Filter Data

Filter prices that only exist in `playstation_games_df`

In [239]:
playstation_prices_df = playstation_prices_df[
    playstation_prices_df['gameid'].isin(playstation_games_df['gameid'])
]

#### Remove Missing Values

Remove prices with missing date acquired:

In [240]:
playstation_prices_df.dropna(subset=['date_acquired'], inplace=True)

Remove prices with empty currency data:

In [241]:
# Keep rows with at least 1 column has data in selected columns
playstation_prices_df.dropna(subset=['usd', 'eur', 'gbp', 'jpy', 'rub'], thresh=1, inplace=True)

View info again:

In [242]:
playstation_prices_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26212 entries, 0 to 62814
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   gameid         26212 non-null  int64  
 1   usd            25203 non-null  float64
 2   eur            23903 non-null  float64
 3   gbp            24758 non-null  float64
 4   jpy            10335 non-null  float64
 5   rub            6156 non-null   float64
 6   date_acquired  26212 non-null  object 
dtypes: float64(5), int64(1), object(1)
memory usage: 1.6+ MB


### Purchased Game Data

#### Info

In [243]:
playstation_purchased_games_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46582 entries, 0 to 46581
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   playerid  46582 non-null  int64 
 1   library   46582 non-null  object
dtypes: int64(1), object(1)
memory usage: 728.0+ KB


#### Trim Data

In [244]:
playstation_purchased_games_df['library'] = \
  playstation_purchased_games_df['library'].str.strip()

#### Filter Data

Filter players that only exist in `playstation_players_df`

In [245]:
playstation_purchased_games_df = playstation_purchased_games_df[
    playstation_purchased_games_df['playerid'].isin(
        playstation_players_df['playerid'])
]

In [246]:
# Convert column to string format
playstation_purchased_games_df['library'] = playstation_purchased_games_df['library'].astype(str)

View info again:

In [247]:
playstation_purchased_games_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46582 entries, 0 to 46581
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   playerid  46582 non-null  int64 
 1   library   46582 non-null  object
dtypes: int64(1), object(1)
memory usage: 728.0+ KB


## Steam Data

### Game Data

#### Info

In [11]:
steam_games_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98248 entries, 0 to 98247
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   gameid               98248 non-null  int64 
 1   title                98245 non-null  object
 2   developers           92689 non-null  object
 3   publishers           92307 non-null  object
 4   genres               92699 non-null  object
 5   supported_languages  92742 non-null  object
 6   release_date         98248 non-null  object
dtypes: int64(1), object(6)
memory usage: 5.2+ MB


#### Trim Data

In [12]:
steam_games_df['title'] = \
  steam_games_df['title'].str.strip()

steam_games_df['developers'] = \
  steam_games_df['developers'].str.strip()

steam_games_df['publishers'] = \
  steam_games_df['publishers'].str.strip()

steam_games_df['genres'] = \
  steam_games_df['genres'].str.strip()

steam_games_df['supported_languages'] = \
  steam_games_df['supported_languages'].str.strip()

steam_games_df['release_date'] = \
  steam_games_df['release_date'].str.strip()

#### Convert Data

`release_date` should be converted to date time type:

In [13]:
steam_games_df['release_date'] = pd.to_datetime(
    steam_games_df['release_date'],
    errors='coerce').dt.strftime('%Y-%m-%d')

#### Remove Duplicates

Check for duplicated games, developers and publishers:

In [14]:
steam_games_df[
    steam_games_df.duplicated(
        subset=['title', 'developers', 'publishers'],
        keep=False
    )
].sort_values(by=['title', 'developers', 'publishers'])

,gameid,title,developers,publishers,genres,supported_languages,release_date
88971,34010,Alpha Protocol™,['Obsidian Entertainment'],['SEGA'],"['Action', 'RPG']","['English', 'French', 'German', 'Italian', 'Sp...",2010-06-01
88972,34019,Alpha Protocol™,['Obsidian Entertainment'],['SEGA'],"['Action', 'RPG']","['English', 'French', 'German', 'Italian', 'Sp...",2010-06-01
24906,2206090,Artifice Playtest,NaN,NaN,NaN,NaN,2022-11-17
56711,2447870,Artifice Playtest,NaN,NaN,NaN,NaN,2023-06-12
69056,2268670,Ascendant Playtest,NaN,NaN,NaN,NaN,2023-01-13
...,...,...,...,...,...,...,...
19764,2076340,蜀山：初章 Playtest,NaN,NaN,NaN,NaN,2023-01-12
19765,2076350,蜀山：初章 Playtest,NaN,NaN,NaN,NaN,2022-07-06
2853,1347240,NaN,NaN,NaN,NaN,['English'],2021-04-20
39705,1116910,NaN,NaN,NaN,"['Action', 'Adventure', 'Casual', 'Indie', 'RP...",NaN,2019-09-25


Remove duplicates:

In [15]:
steam_games_df.drop_duplicates(subset=['title', 'developers', 'publishers'], inplace=True)

#### Remove Missing Values

Remove games with missing title:

In [16]:
steam_games_df.dropna(subset=['title'], inplace=True)

View info again:

In [17]:
steam_games_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 98125 entries, 0 to 98247
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   gameid               98125 non-null  int64 
 1   title                98125 non-null  object
 2   developers           92586 non-null  object
 3   publishers           92201 non-null  object
 4   genres               92590 non-null  object
 5   supported_languages  92633 non-null  object
 6   release_date         98125 non-null  object
dtypes: int64(1), object(6)
memory usage: 6.0+ MB


### Achievement Data

#### Info

In [18]:
steam_achievements_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1939027 entries, 0 to 1939026
Data columns (total 4 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   achievementid  object
 1   gameid         int64 
 2   title          object
 3   description    object
dtypes: int64(1), object(3)
memory usage: 59.2+ MB


#### Trim Data

In [19]:
steam_achievements_df['achievementid'] = \
  steam_achievements_df['achievementid'].str.strip()

steam_achievements_df['title'] = \
  steam_achievements_df['title'].str.strip()

steam_achievements_df['description'] = \
  steam_achievements_df['description'].str.strip()

#### Filter Data

Filter achievements having gameid that only exist in `steam_games_df`

In [20]:
steam_achievements_df = steam_achievements_df[
    steam_achievements_df['gameid'].isin(steam_games_df['gameid'])
]

Filter achievements having id length smaller than 255:

In [21]:
steam_achievements_df = steam_achievements_df[
    steam_achievements_df['achievementid'].str.len() <= 255]

#### About Duplicated Achievements

Steam does not allow duplicate achievements for a single game.

#### Remove Duplicates

Check for achievements with duplicated title and descriptiom:

In [22]:
steam_achievements_df[
    steam_achievements_df.duplicated(
        subset=['title', 'description'],
        keep=False
    )
].sort_values(by=['title', 'description'])

,achievementid,gameid,title,description
144687,806140_NEW_ACHIEVEMENT_61_28,806140,,
144688,806140_NEW_ACHIEVEMENT_61_29,806140,,
144693,806140_NEW_ACHIEVEMENT_62_2,806140,,
144694,806140_NEW_ACHIEVEMENT_62_3,806140,,
144695,806140_NEW_ACHIEVEMENT_62_4,806140,,
...,...,...,...,...
1646881,2206130_NEW_ACHIEVEMENT_1_25,2206130,NaN,NaN
1646882,2206130_NEW_ACHIEVEMENT_1_26,2206130,NaN,NaN
1646883,2206130_NEW_ACHIEVEMENT_1_27,2206130,NaN,NaN
1646884,2206130_NEW_ACHIEVEMENT_1_28,2206130,NaN,NaN


Remove duplicates:

In [23]:
steam_achievements_df.drop_duplicates(subset=['achievementid'], inplace=True)
steam_achievements_df.drop_duplicates(subset=['title', 'description'], inplace=True)

#### Remove Missing Values

Remove achievements with missing title:

In [24]:
steam_achievements_df.dropna(subset=['title'], inplace=True)

View info again:

In [25]:
steam_achievements_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1330877 entries, 0 to 1939026
Data columns (total 4 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   achievementid  1330877 non-null  object
 1   gameid         1330877 non-null  int64 
 2   title          1330877 non-null  object
 3   description    1192444 non-null  object
dtypes: int64(1), object(3)
memory usage: 50.8+ MB


### Player Data

#### Info

In [26]:
steam_players_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 424683 entries, 0 to 424682
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   playerid  424683 non-null  int64 
 1   country   246815 non-null  object
 2   created   377014 non-null  object
dtypes: int64(1), object(2)
memory usage: 9.7+ MB


#### Trim Data

In [27]:
steam_players_df['country'] = \
  steam_players_df['country'].str.strip()

steam_players_df['created'] = \
  steam_players_df['created'].str.strip()

#### Convert Data

`created` should be converted to date time type:

In [28]:
steam_players_df['created'] = pd.to_datetime(
    steam_players_df['created'], errors='coerce')

View info again:

In [29]:
steam_players_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 424683 entries, 0 to 424682
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   playerid  424683 non-null  int64         
 1   country   246815 non-null  object        
 2   created   377014 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 9.7+ MB


### Friend Data

#### Info

In [30]:
steam_friends_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 424683 entries, 0 to 424682
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   playerid  424683 non-null  int64 
 1   friends   339461 non-null  object
dtypes: int64(1), object(1)
memory usage: 6.5+ MB


#### Trim Data

In [31]:
steam_friends_df['friends'] = \
  steam_friends_df['friends'].str.strip()

#### Filter Data

Filter games that only exist in `steam_players_df`

In [32]:
steam_friends_df = steam_friends_df[
    steam_friends_df['playerid'].isin(steam_players_df['playerid'])
]

#### Remove Missing Values

Remove players with no friends:

In [33]:
steam_friends_df.dropna(subset=['friends'], inplace=True)

View info again:

In [34]:
steam_friends_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 339461 entries, 0 to 424682
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   playerid  339461 non-null  int64 
 1   friends   339461 non-null  object
dtypes: int64(1), object(1)
memory usage: 7.8+ MB


### History Data

#### Info

In [35]:
steam_history_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10693879 entries, 0 to 10693878
Data columns (total 3 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   playerid       int64 
 1   achievementid  object
 2   date_acquired  object
dtypes: int64(1), object(2)
memory usage: 244.8+ MB


#### Trim Data

In [36]:
steam_history_df['achievementid'] = \
  steam_history_df['achievementid'].str.strip()

steam_history_df['date_acquired'] = \
  steam_history_df['date_acquired'].str.strip()

#### Convert Data

`date_acquired` should be converted to date time type:

In [37]:
steam_history_df['date_acquired'] = pd.to_datetime(
    steam_history_df['date_acquired'],
    errors='coerce').dt.strftime('%Y-%m-%d %H:%M:%S')

#### Filter Data

Filter history records that only exist in `steam_achievements_df` and `steam_players_df`

In [38]:
steam_history_df = steam_history_df[
    steam_history_df['achievementid'].isin(
        steam_achievements_df['achievementid']) &
    steam_history_df['playerid'].isin(
        steam_players_df['playerid'])
]

View info again:

In [39]:
steam_history_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5362397 entries, 0 to 10693878
Data columns (total 3 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   playerid       int64 
 1   achievementid  object
 2   date_acquired  object
dtypes: int64(1), object(2)
memory usage: 163.6+ MB


### Price Data

#### Info

In [40]:
steam_prices_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4414273 entries, 0 to 4414272
Data columns (total 7 columns):
 #   Column         Dtype  
---  ------         -----  
 0   gameid         int64  
 1   usd            float64
 2   eur            float64
 3   gbp            float64
 4   jpy            float64
 5   rub            float64
 6   date_acquired  object 
dtypes: float64(5), int64(1), object(1)
memory usage: 235.7+ MB


#### Trim Data

In [41]:
steam_prices_df['date_acquired'] = \
  steam_prices_df['date_acquired'].str.strip()

#### Convert Data

`date_acquired` should be converted to date time type:

In [42]:
steam_prices_df['date_acquired'] = pd.to_datetime(
    steam_prices_df['date_acquired'],
    errors='coerce').dt.strftime('%Y-%m-%d %H:%M:%S')

#### Filter Data

Filter prices that only exist in `playstation_games_df`

In [43]:
steam_prices_df = steam_prices_df[
    steam_prices_df['gameid'].isin(steam_games_df['gameid'])
]

#### Remove Missing Values

Remove prices with missing date acquired:

In [44]:
steam_prices_df.dropna(subset=['date_acquired'], inplace=True)

Remove prices with empty currency data:

In [45]:
# Keep rows with at least 1 column has data in selected columns
steam_prices_df.dropna(subset=['usd', 'eur', 'gbp', 'jpy', 'rub'], thresh=1, inplace=True)

View info again:

In [46]:
steam_prices_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3499754 entries, 5 to 4414272
Data columns (total 7 columns):
 #   Column         Dtype  
---  ------         -----  
 0   gameid         int64  
 1   usd            float64
 2   eur            float64
 3   gbp            float64
 4   jpy            float64
 5   rub            float64
 6   date_acquired  object 
dtypes: float64(5), int64(1), object(1)
memory usage: 213.6+ MB


### Purchased Game Data

#### Info

In [47]:
steam_purchased_games_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102548 entries, 0 to 102547
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   playerid  102548 non-null  int64 
 1   library   46941 non-null   object
dtypes: int64(1), object(1)
memory usage: 1.6+ MB


#### Trim Data

In [48]:
steam_purchased_games_df['library'] = \
  steam_purchased_games_df['library'].str.strip()

#### Filter Data

Filter players that only exist in `playstation_players_df`

In [49]:
steam_purchased_games_df = steam_purchased_games_df[
    steam_purchased_games_df['playerid'].isin(
        steam_players_df['playerid'])
]

#### Remove Missing Values

Remove players with missing purchased games:

In [50]:
steam_purchased_games_df.dropna(subset=['library'], inplace=True)

View info again:

In [51]:
steam_purchased_games_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 46941 entries, 0 to 102544
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   playerid  46941 non-null  int64 
 1   library   46941 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.1+ MB


### Review Data

#### Info

In [52]:
steam_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1204534 entries, 0 to 1204533
Data columns (total 8 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   reviewid  1204534 non-null  int64 
 1   playerid  1204534 non-null  int64 
 2   gameid    1204534 non-null  int64 
 3   review    1202437 non-null  object
 4   helpful   1204534 non-null  int64 
 5   funny     1204534 non-null  int64 
 6   awards    1204534 non-null  int64 
 7   posted    1204534 non-null  object
dtypes: int64(6), object(2)
memory usage: 73.5+ MB


#### Trim Data

In [53]:
steam_reviews_df['review'] = \
  steam_reviews_df['review'].str.strip()

steam_reviews_df['posted'] = \
  steam_reviews_df['posted'].str.strip()

#### Convert Data

`posted` should be converted to date time type:

In [54]:
steam_reviews_df['posted'] = pd.to_datetime(
    steam_reviews_df['posted'],
    errors='coerce').dt.strftime('%Y-%m-%d %H:%M:%S')

#### Filter Data

Filter players that only exist in `playstation_players_df`

In [55]:
steam_reviews_df = steam_reviews_df[
    steam_reviews_df['gameid'].isin(
        steam_games_df['gameid']) &
    steam_reviews_df['playerid'].isin(
        steam_players_df['playerid'])
]

#### Remove Missing Values

Remove reviews with no content:

In [56]:
steam_reviews_df.dropna(subset=['review'], inplace=True)

View info again:

In [57]:
steam_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1171765 entries, 0 to 1204533
Data columns (total 8 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   reviewid  1171765 non-null  int64 
 1   playerid  1171765 non-null  int64 
 2   gameid    1171765 non-null  int64 
 3   review    1171765 non-null  object
 4   helpful   1171765 non-null  int64 
 5   funny     1171765 non-null  int64 
 6   awards    1171765 non-null  int64 
 7   posted    1171765 non-null  object
dtypes: int64(6), object(2)
memory usage: 80.5+ MB


## Xbox Data

### Game Data

#### Info

In [9]:
xbox_games_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10489 entries, 0 to 10488
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   gameid               10489 non-null  int64 
 1   title                10489 non-null  object
 2   developers           9913 non-null   object
 3   publishers           9944 non-null   object
 4   genres               9884 non-null   object
 5   supported_languages  3842 non-null   object
 6   release_date         10489 non-null  object
dtypes: int64(1), object(6)
memory usage: 573.7+ KB


#### Trim Data

In [10]:
xbox_games_df['title'] = \
  xbox_games_df['title'].str.strip()

xbox_games_df['developers'] = \
  xbox_games_df['developers'].str.strip()

xbox_games_df['publishers'] = \
  xbox_games_df['publishers'].str.strip()

xbox_games_df['genres'] = \
  xbox_games_df['genres'].str.strip()

xbox_games_df['supported_languages'] = \
  xbox_games_df['supported_languages'].str.strip()

xbox_games_df['release_date'] = \
  xbox_games_df['release_date'].str.strip()

#### Convert Data

`release_date` should be converted to date time type:

In [11]:
xbox_games_df['release_date'] = pd.to_datetime(
    xbox_games_df['release_date'],
    errors='coerce').dt.strftime('%Y-%m-%d %H:%M:%S')

#### Remove Duplicates

Check for games with duplicated title, platform, developers and publishers:

In [12]:
xbox_games_df[
    xbox_games_df.duplicated(
        subset=['title', 'developers', 'publishers'],
        keep=False
    )
].sort_values(by=['title', 'developers', 'publishers'])

,gameid,title,developers,publishers,genres,supported_languages,release_date
1710,465822,2 Synchro Hedgehogs,['grin robot'],['grin robot'],['Platformer'],NaN,2021-12-15 00:00:00
8686,467381,2 Synchro Hedgehogs,['grin robot'],['grin robot'],['Platformer'],NaN,2021-12-15 00:00:00
1300,1600,50 Cent: Blood on the Sand,['Swordfish Studios'],['THQ'],"['Action', 'Adventure', 'shooter']",['Japanese'],2009-02-24 00:00:00
8405,1528,50 Cent: Blood on the Sand,['Swordfish Studios'],['THQ'],"['Action', 'Adventure', 'shooter']","['French', 'German']",2009-02-24 00:00:00
1243,511937,A Winter’s Daydream,NaN,NaN,NaN,NaN,2022-06-24 00:00:00
...,...,...,...,...,...,...,...
5796,16303,iO The Game,['Gamious'],['Game Troopers'],['puzzle'],NaN,2017-02-14 00:00:00
5421,1867,ilomilo,['southend'],['Xbox Game Studios'],['puzzle'],"['French', 'Spanish', 'German', 'Italian']",2011-01-05 00:00:00
9777,1934,ilomilo,['southend'],['Xbox Game Studios'],"['platformer', 'puzzle']","['Japanese', 'French', 'Spanish', 'German', 'I...",2010-11-03 00:00:00
4151,19565,theHunter: Call of the Wild,['Expansive Worlds'],['Avalanche Studios'],['sports'],"['French', 'Spanish', 'German', 'Russian', 'Po...",2017-10-02 00:00:00


Remove duplicates:

In [13]:
xbox_games_df.drop_duplicates(
    subset=['title', 'developers', 'publishers'],
    inplace=True
)

View info again:

In [14]:
xbox_games_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9686 entries, 0 to 10488
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   gameid               9686 non-null   int64 
 1   title                9686 non-null   object
 2   developers           9127 non-null   object
 3   publishers           9157 non-null   object
 4   genres               9101 non-null   object
 5   supported_languages  3414 non-null   object
 6   release_date         9686 non-null   object
dtypes: int64(1), object(6)
memory usage: 605.4+ KB


### Achievement Data

#### Info

In [15]:
xbox_achievements_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 351111 entries, 0 to 351110
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   achievementid  351111 non-null  object 
 1   gameid         351111 non-null  int64  
 2   title          351110 non-null  object 
 3   description    351057 non-null  object 
 4   points         350233 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 13.4+ MB


#### Trim Data

In [16]:
xbox_achievements_df['achievementid'] = \
  xbox_achievements_df['achievementid'].str.strip()

xbox_achievements_df['title'] = \
  xbox_achievements_df['title'].str.strip()

xbox_achievements_df['description'] = \
  xbox_achievements_df['description'].str.strip()

#### Filter Data

Filter games that only exist in `xbox_games_df`

In [17]:
xbox_achievements_df = xbox_achievements_df[
    xbox_achievements_df['gameid'].isin(xbox_games_df['gameid'])
]

#### About Duplicated Achievements

Some Xbox games, particularly those with backwards compatibility, can have duplicate achievement entries, especially when playing an Xbox 360 game on Xbox One or Xbox Series X/S.

#### Remove Missing Values

Remove achievements with missing title:

In [18]:
xbox_achievements_df.dropna(subset=['title'], inplace=True)

View info again:

In [19]:
xbox_achievements_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 296334 entries, 0 to 351110
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   achievementid  296334 non-null  object 
 1   gameid         296334 non-null  int64  
 2   title          296334 non-null  object 
 3   description    296299 non-null  object 
 4   points         295597 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 13.6+ MB


### Player Data

#### Info

In [20]:
xbox_players_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 274450 entries, 0 to 274449
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   playerid  274450 non-null  int64 
 1   nickname  274449 non-null  object
dtypes: int64(1), object(1)
memory usage: 4.2+ MB


#### Trim Data

In [21]:
xbox_players_df['nickname'] = \
  xbox_players_df['nickname'].str.strip()

#### About Duplicated Player Nicknames

All nicknames must be unique.

Check for duplicated nicknames.

In [22]:
xbox_players_df[
    xbox_players_df.duplicated(
        subset=['nickname'],
        keep=False
    )
].sort_values(by=['nickname'])

,playerid,nickname


Looks good.

#### Remove Missing Values

Remove players with missing nickname:

In [23]:
xbox_players_df.dropna(subset=['nickname'], inplace=True)

### History Data

#### Info

In [24]:
xbox_history_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15275900 entries, 0 to 15275899
Data columns (total 3 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   playerid       int64 
 1   achievementid  object
 2   date_acquired  object
dtypes: int64(1), object(2)
memory usage: 349.6+ MB


#### Trim Data

In [25]:
xbox_history_df['achievementid'] = \
  xbox_history_df['achievementid'].str.strip()

xbox_history_df['date_acquired'] = \
  xbox_history_df['date_acquired'].str.strip()

#### Convert Data

`date_acquired` should be converted to date time type:

In [26]:
xbox_history_df['date_acquired'] = pd.to_datetime(
    xbox_history_df['date_acquired'],
    errors='coerce').dt.strftime('%Y-%m-%d %H:%M:%S')

#### Filter Data

Filter history records that only exist in `xbox_achievements_df` and `xbox_players_df`

In [27]:
xbox_history_df = xbox_history_df[
    xbox_history_df['achievementid'].isin(
        xbox_achievements_df['achievementid']) &
    xbox_history_df['playerid'].isin(
        xbox_players_df['playerid'])
]

View info again:

In [28]:
xbox_history_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12240025 entries, 0 to 15275880
Data columns (total 3 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   playerid       int64 
 1   achievementid  object
 2   date_acquired  object
dtypes: int64(1), object(2)
memory usage: 373.5+ MB


### Price Data

#### Info

In [29]:
xbox_prices_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22638 entries, 0 to 22637
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   gameid         22638 non-null  int64  
 1   usd            16003 non-null  float64
 2   eur            15269 non-null  float64
 3   gbp            15812 non-null  float64
 4   jpy            0 non-null      float64
 5   rub            14377 non-null  float64
 6   date_acquired  22638 non-null  object 
dtypes: float64(5), int64(1), object(1)
memory usage: 1.2+ MB


#### Trim Data

In [30]:
xbox_prices_df['date_acquired'] = \
  xbox_prices_df['date_acquired'].str.strip()

#### Convert Data

`date_acquired` should be converted to date time type:

In [31]:
xbox_prices_df['date_acquired'] = pd.to_datetime(
    xbox_prices_df['date_acquired'],
    errors='coerce').dt.strftime('%Y-%m-%d %H:%M:%S')

#### Filter Data

Filter prices that only exist in `xbox_games_df`

In [32]:
xbox_prices_df = xbox_prices_df[
    xbox_prices_df['gameid'].isin(xbox_games_df['gameid'])
]

#### Remove Missing Values

Remove prices with missing date acquired:

In [33]:
xbox_prices_df.dropna(subset=['date_acquired'], inplace=True)

Remove prices with empty currency data:

In [34]:
# Keep rows with at least 1 column has data in selected columns
xbox_prices_df.dropna(subset=['usd', 'eur', 'gbp', 'jpy', 'rub'], thresh=1, inplace=True)

View info again:

In [35]:
xbox_prices_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14770 entries, 0 to 22636
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   gameid         14770 non-null  int64  
 1   usd            14713 non-null  float64
 2   eur            14044 non-null  float64
 3   gbp            14538 non-null  float64
 4   jpy            0 non-null      float64
 5   rub            13219 non-null  float64
 6   date_acquired  14770 non-null  object 
dtypes: float64(5), int64(1), object(1)
memory usage: 923.1+ KB


### Purchased Game Data

#### Info

In [36]:
xbox_purchased_games_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46466 entries, 0 to 46465
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   playerid  46466 non-null  int64 
 1   library   46466 non-null  object
dtypes: int64(1), object(1)
memory usage: 726.2+ KB


#### Trim Data

In [37]:
xbox_purchased_games_df['library'] = \
  xbox_purchased_games_df['library'].str.strip()

#### Filter Data

Filter players that only exist in `xbox_players_df`

In [38]:
xbox_purchased_games_df = xbox_purchased_games_df[
    xbox_purchased_games_df['playerid'].isin(
        xbox_players_df['playerid'])
]

View info again:

In [39]:
xbox_purchased_games_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46466 entries, 0 to 46465
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   playerid  46466 non-null  int64 
 1   library   46466 non-null  object
dtypes: int64(1), object(1)
memory usage: 726.2+ KB


# Define Models

Models are tables in database.

## Base Model

Base model is required.

In [40]:
class Base(DeclarativeBase):
  pass

## PlayStation Models

In [41]:
# Game table
class PlayStationGames(Base):
  # Table name
  __tablename__ = 'playstation_games'

  # Columns
  gameid: Mapped[int] = mapped_column(MEDIUMINT(unsigned=True),
                                      primary_key=True,
                                      nullable=False)
  title: Mapped[str] = mapped_column(NVARCHAR(255), nullable=False)
  platform: Mapped[str] = mapped_column(NVARCHAR(255), nullable=True)
  developers: Mapped[str] = mapped_column(NVARCHAR(255), nullable=True)
  publishers: Mapped[str] = mapped_column(NVARCHAR(255), nullable=True)
  genres: Mapped[str] = mapped_column(NVARCHAR(255), nullable=True)
  supported_languages: Mapped[str] = mapped_column(TEXT, nullable=True)
  release_date: Mapped[datetime] = mapped_column(DATE, nullable=True)

  # Relationships
  achievements: Mapped[List['PlayStationAchievements']] = relationship(
      back_populates='game')
  prices: Mapped[List['PlayStationPrices']] = relationship(
      back_populates='game')


class PlayStationAchievements(Base):
  # Table name
  __tablename__ = 'playstation_achievements'

  # Columns
  # collation: to make the column case sensitive
  # VARCHAR can go with collation setting
  # utf8mb4_bin: allow some special characters (e.g. japanese characters)
  achievementid: Mapped[str] = mapped_column(VARCHAR(255,
                                                     collation="utf8mb4_bin"),
                                             primary_key=True,
                                             nullable=False)
  gameid: Mapped[int] = mapped_column(ForeignKey('playstation_games.gameid'))
  title: Mapped[str] = mapped_column(VARCHAR(255, collation="utf8mb4_bin"), nullable=False)
  description: Mapped[str] = mapped_column(TEXT, nullable=True)
  rarity: Mapped[str] = mapped_column(NVARCHAR(255), nullable=True)

  # Relationships
  game: Mapped['PlayStationGames'] = relationship(back_populates='achievements')
  history: Mapped[List['PlayStationHistory']] = relationship(
      back_populates='achievement')


class PlayStationPlayers(Base):
  # Table name
  __tablename__ = 'playstation_players'

  # Columns
  playerid: Mapped[int] = mapped_column(MEDIUMINT(unsigned=True),
                                        primary_key=True,
                                        nullable=False)
  nickname: Mapped[str] = mapped_column(NVARCHAR(255), nullable=False)
  country: Mapped[str] = mapped_column(NVARCHAR(255), nullable=True)

  # Relationships
  history: Mapped[List['PlayStationHistory']] = relationship(
      back_populates='player')
  purchased_game: Mapped['PlayStationPurchasedGames'] = relationship(
      back_populates='player')


class PlayStationHistory(Base):
  # Table name
  __tablename__ = 'playstation_history'

  # Columns
  playerid: Mapped[int] = mapped_column(
      ForeignKey('playstation_players.playerid'),
      primary_key=True,
      nullable=False)
  achievementid: Mapped[str] = mapped_column(
      ForeignKey('playstation_achievements.achievementid'),
      primary_key=True,
      nullable=False)
  date_acquired: Mapped[datetime] = mapped_column(DATETIME, nullable=True)

  # Relationships
  achievement: Mapped['PlayStationAchievements'] = relationship(
      back_populates='history')
  player: Mapped['PlayStationPlayers'] = relationship(back_populates='history')


class PlayStationPrices(Base):
  # Table name
  __tablename__ = 'playstation_prices'

  # Columns
  gameid: Mapped[int] = mapped_column(ForeignKey('playstation_games.gameid'),
                                      primary_key=True,
                                      nullable=False)
  usd: Mapped[float] = mapped_column(FLOAT, nullable=True)
  eur: Mapped[float] = mapped_column(FLOAT, nullable=True)
  gbp: Mapped[float] = mapped_column(FLOAT, nullable=True)
  jpy: Mapped[float] = mapped_column(FLOAT, nullable=True)
  rub: Mapped[float] = mapped_column(FLOAT, nullable=True)
  date_acquired: Mapped[datetime] = mapped_column(DATE,
                                                  primary_key=True,
                                                  nullable=False)

  # Relationships
  game: Mapped['PlayStationGames'] = relationship(back_populates='prices')


class PlayStationPurchasedGames(Base):
  # Table name
  __tablename__ = 'playstation_purchased_games'

  # Columns
  playerid: Mapped[int] = mapped_column(
      ForeignKey('playstation_players.playerid'),
      primary_key=True,
      nullable=False)
  library: Mapped[str] = mapped_column(MEDIUMTEXT, nullable=True)

  # Relationships
  player: Mapped['PlayStationPlayers'] = relationship(
      back_populates='purchased_game')

## Steam Models

In [42]:
class SteamGames(Base):
  # Table name
  __tablename__ = 'steam_games'

  # Columns
  gameid: Mapped[int] = mapped_column(MEDIUMINT(unsigned=True),
                                      primary_key=True,
                                      nullable=False)
  title: Mapped[str] = mapped_column(VARCHAR(255,
                                             collation="utf8mb4_bin"),
                                     nullable=False)
  developers: Mapped[str] = mapped_column(TEXT, nullable=True)
  publishers: Mapped[str] = mapped_column(VARCHAR(255,
                                             collation="utf8mb4_bin"),
                                          nullable=True)
  genres: Mapped[str] = mapped_column(TEXT, nullable=True)
  supported_languages: Mapped[str] = mapped_column(TEXT, nullable=True)
  release_date: Mapped[datetime] = mapped_column(DATE, nullable=True)

  # Relationships
  achievements: Mapped[List['SteamAchievements']] = relationship(
      back_populates='game')
  prices: Mapped[List['SteamPrices']] = relationship(back_populates='game')
  reviews: Mapped[List['SteamReviews']] = relationship(back_populates='game')


class SteamAchievements(Base):
  # Table name
  __tablename__ = 'steam_achievements'

  # Columns
  achievementid: Mapped[str] = mapped_column(VARCHAR(255,
                                                      collation="utf8mb4_bin"),
                                             primary_key=True,
                                             nullable=False)
  gameid: Mapped[int] = mapped_column(ForeignKey('steam_games.gameid'),
                                      nullable=False)
  title: Mapped[str] = mapped_column(TEXT, nullable=False)
  description: Mapped[str] = mapped_column(TEXT, nullable=True)

  # Relationships
  game: Mapped['SteamGames'] = relationship(back_populates='achievements')
  history: Mapped[List['SteamHistory']] = relationship(
      back_populates='achievement')


class SteamPlayers(Base):
  # Table name
  __tablename__ = 'steam_players'

  # Columns
  # Why bigint? because the largest number is 76561199815552638
  playerid: Mapped[int] = mapped_column(BIGINT,
                                        primary_key=True,
                                        nullable=False)
  country: Mapped[str] = mapped_column(NVARCHAR(255), nullable=True)
  created: Mapped[datetime] = mapped_column(DATETIME, nullable=True)

  # Relationships
  history: Mapped[List['SteamHistory']] = relationship(back_populates='player')
  friend: Mapped['SteamFriends'] = relationship(back_populates='player')
  purchased_game: Mapped['SteamPurchasedGames'] = relationship(
      back_populates='player')
  reviews: Mapped[List['SteamReviews']] = relationship(back_populates='player')


class SteamHistory(Base):
  # Table name
  __tablename__ = 'steam_history'

  # Columns
  playerid: Mapped[int] = mapped_column(ForeignKey('steam_players.playerid'),
                                        primary_key=True,
                                        nullable=False)
  achievementid: Mapped[str] = mapped_column(
      ForeignKey('steam_achievements.achievementid'),
      primary_key=True,
      nullable=False)
  date_acquired: Mapped[datetime] = mapped_column(DATETIME, nullable=True)

  # Relationships
  achievement: Mapped['SteamAchievements'] = relationship(
      back_populates='history')
  player: Mapped['SteamPlayers'] = relationship(back_populates='history')


class SteamPrices(Base):
  # Table name
  __tablename__ = 'steam_prices'

  # Columns
  gameid: Mapped[int] = mapped_column(ForeignKey('steam_games.gameid'),
                                      primary_key=True,
                                      nullable=False)
  usd: Mapped[float] = mapped_column(FLOAT, nullable=True)
  eur: Mapped[float] = mapped_column(FLOAT, nullable=True)
  gbp: Mapped[float] = mapped_column(FLOAT, nullable=True)
  jpy: Mapped[float] = mapped_column(FLOAT, nullable=True)
  rub: Mapped[float] = mapped_column(FLOAT, nullable=True)
  date_acquired: Mapped[datetime] = mapped_column(DATE,
                                                  primary_key=True,
                                                  nullable=False)

  # Relationships
  game: Mapped['SteamGames'] = relationship(back_populates='prices')


class SteamFriends(Base):
  # Table name
  __tablename__ = 'steam_friends'

  # Columns
  playerid: Mapped[int] = mapped_column(ForeignKey('steam_players.playerid'),
                                        primary_key=True,
                                        nullable=False)
  friends: Mapped[str] = mapped_column(TEXT, nullable=False)

  # Relationships
  player: Mapped['SteamPlayers'] = relationship(back_populates='friend')


class SteamPurchasedGames(Base):
  # Table name
  __tablename__ = 'steam_purchased_games'

  # Columns
  playerid: Mapped[int] = mapped_column(ForeignKey('steam_players.playerid'),
                                        primary_key=True,
                                        nullable=False)
  # Why MEDIUMTEXT? There is a row with 265945 characters
  library: Mapped[str] = mapped_column(MEDIUMTEXT,
                                       nullable=False)

  # Relationships
  player: Mapped['SteamPlayers'] = relationship(back_populates='purchased_game')


class SteamReviews(Base):
  # Table name
  __tablename__ = 'steam_reviews'

  # Columns
  reviewid: Mapped[int] = mapped_column(MEDIUMINT(unsigned=True),
                                        primary_key=True,
                                        nullable=False)
  playerid: Mapped[int] = mapped_column(ForeignKey('steam_players.playerid'),
                                        nullable=False)
  gameid: Mapped[int] = mapped_column(ForeignKey('steam_games.gameid'),
                                      nullable=False)
  review: Mapped[str] = mapped_column(TEXT, nullable=False)
  helpful: Mapped[int] = mapped_column(SMALLINT(unsigned=True), nullable=True)
  funny: Mapped[int] = mapped_column(SMALLINT(unsigned=True), nullable=True)
  awards: Mapped[int] = mapped_column(SMALLINT(unsigned=True), nullable=True)
  posted: Mapped[datetime] = mapped_column(DATE, nullable=True)

  # Relationships
  player: Mapped['SteamPlayers'] = relationship(back_populates='reviews')
  game: Mapped['SteamGames'] = relationship(back_populates='reviews')

## Xbox Models

In [43]:
# class XboxGames(Base):
#   # Table name
#   __tablename__ = 'xbox_games'
#
#   # Columns
#   gameid: Mapped[int] = mapped_column(MEDIUMINT(unsigned=True),
#                                       primary_key=True,
#                                       nullable=False)
#   title: Mapped[str] = mapped_column(VARCHAR(255,
#                                              collation="utf8mb4_bin"),
#                                      nullable=False)
#   developers: Mapped[str] = mapped_column(NVARCHAR(255), nullable=True)
#   publishers: Mapped[str] = mapped_column(NVARCHAR(255), nullable=True)
#   genres: Mapped[str] = mapped_column(NVARCHAR(255), nullable=True)
#   supported_languages: Mapped[str] = mapped_column(NVARCHAR(255), nullable=True)
#   release_date: Mapped[datetime] = mapped_column(DATE, nullable=True)
#
#   # Relationships
#   achievements: Mapped[List['XboxAchievements']] = relationship(
#       back_populates='game')
#   prices: Mapped[List['XboxPrices']] = relationship(back_populates='game')
#
#
# class XboxAchievements(Base):
#   # Table name
#   __tablename__ = 'xbox_achievements'
#
#   # Columns
#   achievementid: Mapped[str] = mapped_column(VARCHAR(255,
#                                                      collation="utf8mb4_bin"),
#                                              primary_key=True,
#                                              nullable=False)
#   gameid: Mapped[int] = mapped_column(ForeignKey('xbox_games.gameid'),
#                                       nullable=False)
#   title: Mapped[str] = mapped_column(VARCHAR(255,
#                                              collation="utf8mb4_bin"),
#                                      nullable=False)
#   description: Mapped[str] = mapped_column(VARCHAR(255,
#                                                    collation="utf8mb4_bin"),
#                                            nullable=True)
#   points: Mapped[float] = mapped_column(FLOAT, nullable=True)
#
#   # Relationships
#   game: Mapped['XboxGames'] = relationship(back_populates='achievements')
#   history: Mapped[List['XboxHistory']] = relationship(
#       back_populates='achievement')
#
#
# class XboxPlayers(Base):
#   # Table name
#   __tablename__ = 'xbox_players'
#
#   # Columns
#   playerid: Mapped[int] = mapped_column(INTEGER,
#                                         primary_key=True,
#                                         nullable=False)
#   nickname: Mapped[str] = mapped_column(NVARCHAR(255),
#                                         nullable=False)
#
#   # Relationships
#   history: Mapped[List['XboxHistory']] = relationship(back_populates='player')
#   purchased_game: Mapped['XboxPurchasedGames'] = relationship(
#       back_populates='player')
#
#
# class XboxHistory(Base):
#   # Table name
#   __tablename__ = 'xbox_history'
#
#   # Columns
#   playerid: Mapped[int] = mapped_column(ForeignKey('xbox_players.playerid'),
#                                         primary_key=True,
#                                         nullable=False)
#   achievementid: Mapped[str] = mapped_column(
#       ForeignKey('xbox_achievements.achievementid'),
#       primary_key=True,
#       nullable=False)
#   date_acquired: Mapped[datetime] = mapped_column(DATETIME, nullable=True)
#
#   # Relationships
#   achievement: Mapped['XboxAchievements'] = relationship(
#       back_populates='history')
#   player: Mapped['XboxPlayers'] = relationship(back_populates='history')
#
#
# class XboxPrices(Base):
#   # Table name
#   __tablename__ = 'xbox_prices'
#
#   # Columns
#   gameid: Mapped[int] = mapped_column(ForeignKey('xbox_games.gameid'),
#                                       primary_key=True,
#                                       nullable=False)
#   usd: Mapped[float] = mapped_column(FLOAT, nullable=True)
#   eur: Mapped[float] = mapped_column(FLOAT, nullable=True)
#   gbp: Mapped[float] = mapped_column(FLOAT, nullable=True)
#   jpy: Mapped[float] = mapped_column(FLOAT, nullable=True)
#   rub: Mapped[float] = mapped_column(FLOAT, nullable=True)
#   date_acquired: Mapped[datetime] = mapped_column(DATE,
#                                                   primary_key=True,
#                                                   nullable=False)
#
#   # Relationships
#   game: Mapped['XboxGames'] = relationship(back_populates='prices')
#
#
# class XboxPurchasedGames(Base):
#   # Table name
#   __tablename__ = 'xbox_purchased_games'
#
#   # Columns
#   playerid: Mapped[int] = mapped_column(ForeignKey('xbox_players.playerid'),
#                                         primary_key=True,
#                                         nullable=False)
#   library: Mapped[str] = mapped_column(TEXT,
#                                        nullable=False)
#
#   # Relationships
#   player: Mapped['XboxPlayers'] = relationship(back_populates='purchased_game')

# Database Info

Note: feel free to update these fields to match your server connection:

In [45]:
server_host = '127.0.0.1'
server_port = 3306
server_user = 'root'
server_password = '123456'
database_name = 'pixelsales'

# Create Database

In [41]:
mysql_connection = mysql.connector.connect(
    host=server_host,
    port=server_port,
    user=server_user,
    password=server_password)

cursor = mysql_connection.cursor()
cursor.execute(f'''CREATE DATABASE IF NOT EXISTS {database_name}''')

cursor.close()
mysql_connection.close()

# Create Engine

In [46]:
# Define connection url
database_url = f'''mysql+mysqlconnector://{server_user}:{server_password}@{server_host}:{server_port}/{database_name}'''
engine = create_engine(database_url)

# Create Tables In Database

In [47]:
# Create tables if not exist.
Base.metadata.create_all(engine)

# Import Data

Define a function to import data:

In [49]:
# chunksize=n: The number of rows in each batch to be written at a time.
def insert_data_from_df(df, table_name, chunksize=10000):
  # Create a session
  with Session(engine) as session:
    try:
      df.to_sql(table_name,
                con=engine,
                # Drop table if redo the insert operator
                if_exists="append",
                index=False,
                chunksize=chunksize,
                # Pass multiple values in a single INSERT clause.
                method="multi")
      print(f'Successfully inserted data into {table_name}')
    except Exception as exception:
      print(f'Couldn\'t insert data to database. Error: {exception}')
    finally:
      # Close session
      session.close()

## PlayStation Data

In [ ]:
insert_data_from_df(playstation_games_df, PlayStationGames.__tablename__)

In [350]:
insert_data_from_df(playstation_achievements_df, PlayStationAchievements.__tablename__)

Successfully inserted data into playstation_achievements


In [351]:
insert_data_from_df(playstation_players_df, PlayStationPlayers.__tablename__)

Successfully inserted data into playstation_players


In [352]:
insert_data_from_df(playstation_history_df, PlayStationHistory.__tablename__)

Successfully inserted data into playstation_history


In [353]:
insert_data_from_df(playstation_prices_df, PlayStationPrices.__tablename__)

Successfully inserted data into playstation_prices


In [354]:
insert_data_from_df(playstation_purchased_games_df,
                    PlayStationPurchasedGames.__tablename__,
                    100)

Successfully inserted data into playstation_purchased_games


## Steam Data

In [383]:
insert_data_from_df(steam_games_df, SteamGames.__tablename__)

Successfully inserted data into steam_games


In [384]:
insert_data_from_df(steam_achievements_df, SteamAchievements.__tablename__)

Successfully inserted data into steam_achievements


In [385]:
insert_data_from_df(steam_players_df, SteamPlayers.__tablename__)

Successfully inserted data into steam_players


In [386]:
insert_data_from_df(steam_history_df, SteamHistory.__tablename__)

Successfully inserted data into steam_history


In [387]:
insert_data_from_df(steam_prices_df, SteamPrices.__tablename__)

Successfully inserted data into steam_prices


In [389]:
insert_data_from_df(steam_purchased_games_df, SteamPurchasedGames, 100)

Successfully inserted data into steam_purchased_games


In [71]:
insert_data_from_df(steam_friends_df, SteamFriends.__tablename__, 50)

Successfully inserted data into steam_friends


In [72]:
insert_data_from_df(steam_reviews_df, SteamReviews.__tablename__, 50)

Successfully inserted data into steam_reviews


## Xbox Data

In [140]:
# insert_data_from_df(xbox_games_df, XboxGames.__tablename__)

Successfully inserted data into xbox_games


In [159]:
# insert_data_from_df(xbox_achievements_df, XboxAchievements.__tablename__)

Successfully inserted data into xbox_achievements


In [162]:
# insert_data_from_df(xbox_players_df, XboxPlayers.__tablename__)

Successfully inserted data into xbox_players


In [50]:
# insert_data_from_df(xbox_history_df, XboxHistory.__tablename__)

Successfully inserted data into xbox_history


In [51]:
# insert_data_from_df(xbox_prices_df, XboxPrices.__tablename__)

Successfully inserted data into xbox_prices


In [52]:
# insert_data_from_df(xbox_purchased_games_df,
#                     XboxPurchasedGames.__tablename__,
#                     100)

Successfully inserted data into xbox_purchased_games


# Close engine

In [53]:
# Close the engine when done
engine.dispose()